# Importing Libraries

In [9]:
import scapy.all as scapy
import numpy as np
import joblib
import pandas as pd

# Code

In [10]:
df = pd.read_csv('Dataset/ML-EdgeIIOT-dataset-cleaned-2.csv')

In [11]:
# Create a summary DataFrame with data types
summary_df = pd.DataFrame({
    "Column Name": df.columns,
    "Data Type": df.dtypes.values,   # ✅ Include data types
    "Min Value": df.min(numeric_only=True),
    "Max Value": df.max(numeric_only=True),
    "Unique Values": df.nunique()
}).reset_index(drop=True)

# Print the summary DataFrame
summary_df


,Column Name,Data Type,Min Value,Max Value,Unique Values
0,arp.hw.size,int64,0.0,6.000000e+00,2
1,icmp.checksum,float64,0.0,6.553200e+04,13187
2,icmp.seq_le,float64,0.0,6.552400e+04,13824
3,http.content_length,float64,0.0,8.365500e+04,33
4,http.request.method,int64,0.0,4.000000e+00,5
5,http.referer,int64,0.0,2.000000e+00,3
6,http.response,int64,0.0,1.000000e+00,2
7,tcp.ack,float64,0.0,2.147333e+09,27929
8,tcp.ack_raw,float64,0.0,4.294947e+09,94716
9,tcp.dstport,float64,0.0,6.553500e+04,23188


In [4]:
rfe = joblib.load("Models/rfe_selector.pkl")
gb_model = joblib.load("Models/gradient_boosting.pkl")

# Attack Mapping
attack_mapping = {
    'MITM': 1, 'Fingerprinting': 2, 'Ransomware': 3, 'Uploading': 4,
    'SQL_injection': 5, 'DDoS_HTTP': 6, 'DDoS_TCP': 7, 'Password': 8,
    'Port_Scanning': 9, 'Vulnerability_scanner': 10, 'Backdoor': 11,
    'XSS': 12, 'Normal': 13, 'DDoS_UDP': 14, 'DDoS_ICMP': 15
}

In [ ]:
# Define attack-specific packet characteristics
def generate_packet(attack_type):
    packet = {
        "arp.hw.size": 6,  # Default valid ARP size
        "icmp.checksum": 12345,  # Normal checksum range
        "icmp.seq_le": 54321,
        "http.content_length": 0,
        "http.request.method": 0,
        "http.referer": 0,
        "http.response": 0,
        "tcp.ack": 0,
        "tcp.ack_raw": 0,
        "tcp.dstport": 80,  # Default HTTP port
        "tcp.flags.ack": 1,
        "tcp.len": 0,
        "tcp.seq": 0,
        "udp.port": 0,
        "udp.stream": 0,
        "udp.time_delta": 0,
        "dns.qry.name": 0,
        "dns.qry.qu": 0,
        "dns.retransmission": 0,
        "dns.retransmit_request": 0,
        "mqtt.msgtype": 0,
        "mqtt.topic_len": 0,
        "mqtt.ver": 4,  # MQTT version 4
        "ip.src_1": 192,  # Default IP structure
        "ip.src_2": 168,
        "ip.src_3": 1,
        "ip.src_4": 10,
        "ip.dst_1": 192,
        "ip.dst_2": 168,
        "ip.dst_3": 1,
        "ip.dst_4": 1,
        "tcp_flag_category": 1,
        "Attack_label": 1,
        "Attack_type": attack_type
    }
    
    if attack_type == "MITM":
        packet.update({
            "tcp.ack": np.random.uniform(1e9, 2.147e9),
            "dns.retransmission": 10,  # High retransmissions
            "ip.dst_1": 10,  # Destination could be faked
            "tcp.flags.ack": 0,  # Unexpected flag behavior
        })

    elif attack_type == "DDoS_HTTP":
        packet.update({
            "http.content_length": np.random.uniform(50000, 80000),  # Large request size
            "tcp.dstport": 80,  # Attack on HTTP port
            "tcp.len": np.random.uniform(1000, 65000),
        })

    elif attack_type == "SQL_injection":
        packet.update({
            "dns.qry.name": np.random.uniform(1e6, 2.8e6),  # Fake DNS query
            "tcp.dstport": 1433,  # SQL Server port
            "tcp.flags.ack": 1,
        })

    elif attack_type == "Port_Scanning":
        packet.update({
            "tcp.dstport": np.random.choice([22, 25, 53, 80, 443, 3306]),  # Scanning common ports
            "tcp.seq": np.random.uniform(1e8, 2e8),
        })

    elif attack_type == "DDoS_TCP":
        packet.update({
            "tcp.ack_raw": np.random.uniform(1e9, 4.2e9),
            "tcp.flags.ack": 0,  # Flooding without ACK
            "tcp.len": np.random.uniform(2000, 60000),
        })

    elif attack_type == "DDoS_UDP":
        packet.update({
            "udp.port": np.random.choice([53, 123, 161, 500]),  # UDP ports used in amplification attacks
            "udp.stream": np.random.uniform(1e5, 2.8e6),
        })

    elif attack_type == "XSS":
        packet.update({
            "http.referer": 1,  # Injection in referer field
            "http.request.method": 2,  # POST method used in XSS attacks
            "tcp.len": np.random.uniform(1000, 5000),
        })

    elif attack_type == "Fingerprinting":
        packet.update({
            "tcp_flag_category": 4,  # Uncommon flag categories used for fingerprinting
            "dns.qry.qu": 1,  # Unusual DNS query behavior
        })

    elif attack_type == "Password":
        packet.update({
            "mqtt.msgtype": 10,  # Authentication message
            "mqtt.topic_len": 20,  # Suspiciously long topic length
        })

    elif attack_type == "Ransomware":
        packet.update({
            "tcp.dstport": 445,  # SMB port used for ransomware attacks
            "dns.qry.name": np.random.uniform(1.5e6, 2.8e6),  # Suspicious DNS activity
        })

    elif attack_type == "Vulnerability_scanner":
        packet.update({
            "tcp.seq": np.random.uniform(1e8, 2e8),
            "tcp.dstport": np.random.choice([135, 139, 445]),  # Windows vulnerability ports
        })

    elif attack_type == "Backdoor":
        packet.update({
            "tcp.dstport": 4444,  # Common backdoor port
            "mqtt.ver": 3,  # Older MQTT version used in exploits
        })

    elif attack_type == "Normal":
        packet.update({
            "tcp.ack": np.random.uniform(1e6, 5e6),
            "tcp.dstport": 443,  # HTTPS communication
            "dns.qry.name": np.random.uniform(1e5, 5e5),  # Regular browsing activity
        })

    return packet


# Generate attack packets for all attack types
attack_types = [
    "MITM", "Fingerprinting", "Ransomware", "Uploading",
    "SQL_injection", "DDoS_HTTP", "DDoS_TCP", "Password",
    "Port_Scanning", "Vulnerability_scanner", "Backdoor",
    "XSS", "Normal", "DDoS_UDP", "DDoS_ICMP"
]

attack_packets = [generate_packet(attack) for attack in attack_types]

# Convert to DataFrame
attack_df = pd.DataFrame(attack_packets)

# Show the first few rows
print(attack_df)

    arp.hw.size  icmp.checksum  icmp.seq_le  http.content_length  \
0             6          12345        54321             0.000000   
1             6          12345        54321             0.000000   
2             6          12345        54321             0.000000   
3             6          12345        54321             0.000000   
4             6          12345        54321             0.000000   
5             6          12345        54321         67992.963208   
6             6          12345        54321             0.000000   
7             6          12345        54321             0.000000   
8             6          12345        54321             0.000000   
9             6          12345        54321             0.000000   
10            6          12345        54321             0.000000   
11            6          12345        54321             0.000000   
12            6          12345        54321             0.000000   
13            6          12345        54321     

In [13]:
def predict_attack(new_packet):
    """Predict attack type using RFE feature selection + Gradient Boosting model."""
    
    # Extract features from new packet (ensure it's a 1D NumPy array)
    X_new = np.array(extract_features(new_packet)).reshape(1, -1)  # Reshape for model compatibility

    # Apply RFE feature selection
    X_selected = rfe.transform(X_new)  # Ensure input is 2D

    # Predict using Gradient Boosting model
    prediction = gb_model.predict(X_selected)[0]

    return prediction

In [ ]:
for _, row in attack_df.iterrows():
    print(f"Predicted Attack: {predict_attack(row)}")

TypeError: unsupported operand type(s) for &: 'Flags' and 'int'